In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)

In [4]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [5]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [6]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [7]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step),
          'Cost: %3f' % sess.run(cost,feed_dict={X:x_data, Y:y_data}))
    
    summary = sess.run(merged, feed_dict = {X:x_data,Y:y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step: 1,  Cost: 1.529901
Step: 2,  Cost: 1.427296
Step: 3,  Cost: 1.334983
Step: 4,  Cost: 1.253266
Step: 5,  Cost: 1.181904
Step: 6,  Cost: 1.119882
Step: 7,  Cost: 1.065367
Step: 8,  Cost: 1.015969
Step: 9,  Cost: 0.971797
Step: 10,  Cost: 0.931336
Step: 11,  Cost: 0.893970
Step: 12,  Cost: 0.859395
Step: 13,  Cost: 0.827476
Step: 14,  Cost: 0.798170
Step: 15,  Cost: 0.771459
Step: 16,  Cost: 0.747237
Step: 17,  Cost: 0.724596
Step: 18,  Cost: 0.704458
Step: 19,  Cost: 0.686511
Step: 20,  Cost: 0.670511
Step: 21,  Cost: 0.656265
Step: 22,  Cost: 0.643694
Step: 23,  Cost: 0.632553
Step: 24,  Cost: 0.622639
Step: 25,  Cost: 0.613848
Step: 26,  Cost: 0.606235
Step: 27,  Cost: 0.599797
Step: 28,  Cost: 0.594082
Step: 29,  Cost: 0.589029
Step: 30,  Cost: 0.584627
Step: 31,  Cost: 0.580767
Step: 32,  Cost: 0.577374
Step: 33,  Cost: 0.574500
Step: 34,  Cost: 0.571935
Step: 35,  Cost: 0.569646
Step: 36,  Cost: 0.567610
Step: 37,  Cost: 0.565797
Step: 38,  Cost: 0.564186
Step: 39,  Cost: 0.56

In [8]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-100'

In [9]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print("예측값:", sess.run(prediction,feed_dict={X: x_data}))
print("실제값:", sess.run(target,feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.000000
